## Timing Comparison for Training Loops

In [1]:
import wandb
import config
import diamond_world_model_trainer as trainer
import incremental_training as incremental_trainer
import os
import shutil
import time
import gc
import torch

run = wandb.init(project='timing-comparison', reinit=True)

Imports successful.
Visualization helpers defined.
Training and validation epoch functions adapted for Batch object and Denoiser.forward.


wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: irvin-hwang (irvin-hwang-simulacra-systems) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


### Run `train_diamond_model` on incremental dataset

In [2]:
gc.collect()
torch.cuda.empty_cache()

config.OUTPUT_DIR = os.path.join(config.AUXILIARY_DIR, 'output_model_2hz_DIAMOND_laundry_incremental_test')
config.DATA_DIR = os.path.join(config.AUXILIARY_DIR, 'jetbot_data_two_actions_incremental_test')
config.IMAGE_DIR = os.path.join(config.DATA_DIR, 'images')
config.CSV_PATH = os.path.join(config.DATA_DIR, 'laundry_data_incremental_test.csv')
config.EARLY_STOPPING_PATIENCE = 1

if os.path.exists(config.OUTPUT_DIR):
    shutil.rmtree(config.OUTPUT_DIR)
os.makedirs(config.OUTPUT_DIR, exist_ok=True)

wandb.init(project='timing-comparison', reinit=True)
start = time.time()
incremental_trainer.main()
inc_duration = time.time() - start
inc_hist = run_inc.history(keys=["incremental_fwbw_reserved"]) or {}
inc_peak_reserved = None
if hasattr(inc_hist, "__getitem__"):
    try:
        inc_peak_reserved = inc_hist["incremental_fwbw_reserved"].max()
    except Exception:
        inc_peak_reserved = None
wandb.finish()

Found 1 new sessions to collect.


Wrote 4538 entries to C:\Projects\jetbot-diffusion-world-model-kong-finder-aux\jetbot_new_data_incremental_test\new.csv
Loaded combined CSV with columns: ['session_id', 'image_path', 'timestamp', 'action']
Fresh dataset size is 299.
Loaded combined CSV with columns: ['session_id', 'image_path', 'timestamp', 'action']
Full dataset size: 292
Creating new train/val split...
Saved new dataset split to C:\Projects\jetbot-diffusion-world-model-kong-finder-aux\output_model_2hz_DIAMOND_laundry_incremental_test\dataset_split.pth
[incremental_before_dataloader] GPU mem alloc: 0.0 MB, reserved: 0.0 MB
[incremental_after_dataloader] GPU mem alloc: 0.0 MB, reserved: 0.0 MB
Starting training
Using device: cuda
Starting from fresh model
Incremental training for at least 320 and at most 320000 with 320 number of steps per fresh data epoch
[Incremental Pre training loop] GPU mem alloc: 1263.9 MB, reserved: 1286.0 MB


Incremental Training Steps:   0%|          | 0/320000 [00:00<?, ?it/s]

Filtering dataset with 30 samples for actions: [0.0]


Filtering Dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Filtered down to 13 samples.
Filtering dataset with 30 samples for actions: [0.13]


Filtering Dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Filtered down to 17 samples.
[Incremental post batch fetch] GPU mem alloc: 1263.9 MB, reserved: 1286.0 MB
[incremental_step0_after_move] GPU mem alloc: 1264.4 MB, reserved: 1288.0 MB
[Incremental post fw/bw] GPU mem alloc: 2550.7 MB, reserved: 2638.0 MB
[Incremental post batch fetch] GPU mem alloc: 2550.7 MB, reserved: 2638.0 MB
[incremental_step1_after_move] GPU mem alloc: 2550.7 MB, reserved: 2638.0 MB
[Incremental post fw/bw] GPU mem alloc: 2550.7 MB, reserved: 3588.0 MB
[Incremental post batch fetch] GPU mem alloc: 2550.7 MB, reserved: 3588.0 MB
[incremental_step2_after_move] GPU mem alloc: 2550.7 MB, reserved: 3588.0 MB
[Incremental post fw/bw] GPU mem alloc: 2550.7 MB, reserved: 3588.0 MB
[Incremental post batch fetch] GPU mem alloc: 2550.7 MB, reserved: 3588.0 MB
[incremental_step3_after_move] GPU mem alloc: 2550.7 MB, reserved: 3588.0 MB
[Incremental post fw/bw] GPU mem alloc: 2550.7 MB, reserved: 3588.0 MB
[Incremental post batch fetch] GPU mem alloc: 2550.7 MB, reserved: 3588

KeyboardInterrupt: 

### Run `_main_training` on non-incremental dataset

In [ ]:
gc.collect()
torch.cuda.empty_cache()

config.OUTPUT_DIR = os.path.join(config.AUXILIARY_DIR, 'output_model_2hz_DIAMOND_laundry_nonincremental_test')
config.DATA_DIR = os.path.join(config.AUXILIARY_DIR, 'jetbot_data_two_actions_nonincremental_test')
config.IMAGE_DIR = os.path.join(config.DATA_DIR, 'images')
config.CSV_PATH = os.path.join(config.DATA_DIR, 'laundry_data_incremental_test.csv')
config.NUM_EPOCHS = 1

if os.path.exists(config.OUTPUT_DIR):
    shutil.rmtree(config.OUTPUT_DIR)
os.makedirs(config.OUTPUT_DIR, exist_ok=True)

start = time.time()
trainer_run = trainer._main_training(finish_run=False)
noninc_duration = time.time() - start
noninc_hist = trainer_run.history(keys=["nonincremental_fwbw_reserved"]) or {}
noninc_peak_reserved = None
if hasattr(noninc_hist, "__getitem__"):
    try:
        noninc_peak_reserved = noninc_hist["nonincremental_fwbw_reserved"].max()
    except Exception:
        noninc_peak_reserved = None
wandb.finish()

### Compare timings

In [ ]:
import pandas as pd

comparison_df = pd.DataFrame({
    'run': ['non_incremental', 'incremental'],
    'duration_sec': [noninc_duration, inc_duration],
    'peak_reserved_mb': [noninc_peak_reserved, inc_peak_reserved]
})
comparison_df